**CLTV Prediction with BG-NBD and Gamma-Gamma**

In [1]:
#!pip install lifetimes
#import pandas as pd
#import datetime as dt
#import matplotlib.pyplot as plt
#from lifetimes import BetaGeoFitter
#from lifetimes import GammaGammaFitter
#from lifetimes.plotting import plot_period_transactions
#from sklearn.preprocessing import MinMaxScaler
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', 500)
#pd.set_option('display.float_format', lambda x: '%.4f' % x)

#df_ = pd.read_csv("datasets/flo_data_20k.csv")

*Outlier Functions*

In [2]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range

    if dataframe[variable].dtype != 'O':
        up_limit = round(up_limit)
        low_limit = round(low_limit)

    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    #dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit
    

#replace_with_thresholds(df, "order_num_total_ever_online")
#replace_with_thresholds(df, "order_num_total_ever_offline")

*Creating CLTV DF*

In [3]:
#df["last_order_date"].max()

#today_date = dt.datetime(2021, 6, 1)

#cltv_df = df.groupby('master_id').agg({
#   'last_order_date': [lambda x: (x.max() - df.loc[x.index, 'first_order_date'].min()).days,
#                       lambda x: (today_date - df.loc[x.index, 'first_order_date'].min()).days],
#    'total_purc': lambda x: x.sum(),
#    'total_spending': lambda x: x.sum()})

#cltv_df.columns = cltv_df.columns.droplevel(0) ##sütun üstündeki ilk sırayı siliyoruz
#cltv_df.columns = ["recency", "T", "frequency", "monetary"]


#### WEEKLY PERIOD ###

#cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"] 

#cltv_df = cltv_df[(cltv_df["frequency"] > 1)] 

#cltv_df["recency"] = cltv_df["recency"] / 7 

#cltv_df["T"] = cltv_df["T"] / 7

*Applying BG/NBD Model*

In [4]:
#bgf = BetaGeoFitter(penalizer_coef=0.001)

#bgf.fit(cltv_df['frequency'],
#        cltv_df['recency'],
#        cltv_df['T'])


#cltv_df["exp_sales_3_month"] = bgf.predict(4,
#                                              cltv_df['frequency'],
#                                              cltv_df['recency'],
#                                              cltv_df['T'])

*Applying Gamma-Gamma Model*

In [5]:
#ggf = GammaGammaFitter(penalizer_coef=0.01)

#ggf.fit(cltv_df['frequency'], cltv_df['monetary'])

#cltv_df["exp_average_value"] = ggf.conditional_expected_average_profit(cltv_df['frequency'], cltv_df['monetary'])


*(1 Month) CLTV*

In [6]:
#cltv = ggf.customer_lifetime_value(bgf,
#                                   cltv_df['frequency'],
#                                   cltv_df['recency'],
#                                   cltv_df['T'],
#                                   cltv_df['monetary'],
#                                   time=1, 
#                                   freq="W",
#                                   discount_rate=0.01)

*Creating CLTV Segments*

In [7]:
#cltv["cltv_segment"] = pd.qcut(cltv["clv"], 4, labels=["D", "C", "B", "A"])